In [1]:
import pyspark
import pandas as pd
from pydataset import data
spark = pyspark.sql.SparkSession.builder.getOrCreate()


### Create a spark data frame that contains your favorite programming languages.

#### The name of the column should be language\
#### View the schema of the dataframe\
#### Output the shape of the dataframe\
#### Show the first 5 records in the dataframe

In [586]:
pd_df = pd.DataFrame({'language': ['Python', 'C', 'SQL']}, 
                     index = [1, 2, 3])

In [587]:
pd_df

,language
1,Python
2,C
3,SQL


In [588]:
sp_df = spark.createDataFrame(pd_df)
sp_df

DataFrame[language: string]

In [589]:
sp_df.show()

+--------+
|language|
+--------+
|  Python|
|       C|
|     SQL|
+--------+



In [590]:
sp_df.describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|       3|
|   mean|    null|
| stddev|    null|
|    min|       C|
|    max|     SQL|
+-------+--------+



In [592]:
len(sp_df.columns)

1

In [7]:
sp_df.show(2)

+--------+
|language|
+--------+
|  Python|
|       C|
+--------+
only showing top 2 rows



### Load the mpg dataset as a spark dataframe.

#### Create 1 column of output that contains a message like the one below:
The 1999 audi a4 has a 4 cylinder engine.

#### For each vehicle.
Transform the trans column so that it only contains either manual or auto.

In [8]:
mpg_pd = data('mpg')

In [9]:
mpg_pd.head(4)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact


In [10]:
mpg = spark.createDataFrame(mpg_pd)

In [11]:
mpg.show(4)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 4 rows



In [12]:
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_extract, regexp_replace

In [13]:
col1 = mpg.select(concat(lit('The '), mpg.year,lit(' '), mpg.manufacturer,lit(' '), mpg.model, lit(' has a '), mpg.cyl, lit(' cylinder engine'))).show(5,truncate=False)

+-----------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine)|
+-----------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
+-----------------------------------------------------------------------------+
only showing top 5 rows



In [14]:
mpg.select("trans",
    regexp_extract("trans", r"^(\w+)", 1).alias("ntrans")).show(5)

+----------+------+
|     trans|ntrans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 5 rows



Load the tips dataset as a spark dataframe.

What percentage of observations are smokers?\
Create a column that contains the tip percentage\
Calculate the average tip percentage for each combination of sex and smoker.

In [15]:
tips_pd = data('tips')

In [16]:
tips = spark.createDataFrame(tips_pd)

In [17]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [18]:
pct = sum(tips.filter(tips.smoker == 'Yes').smoker)/count(tips.smoker)

In [19]:
a = tips.filter(tips.smoker == 'Yes')

In [20]:
a.select(count(a.smoker)).show()

+-------------+
|count(smoker)|
+-------------+
|           93|
+-------------+



In [21]:
tips.select(count(tips.smoker)).show()

+-------------+
|count(smoker)|
+-------------+
|          244|
+-------------+



Create a column that contains the tip percentage

In [593]:
tips_pct = col('tip')/ col('total_bill')*100
tips.select(tips.total_bill, tips.tip, tips_pct.alias('tip_pcnt'), tips.sex, tips.smoker).show(3)

+----------+----+------------------+------+------+
|total_bill| tip|          tip_pcnt|   sex|smoker|
+----------+----+------------------+------+------+
|     16.99|1.01|5.9446733372572105|Female|    No|
|     10.34|1.66|16.054158607350097|  Male|    No|
|     21.01| 3.5|16.658733936220845|  Male|    No|
+----------+----+------------------+------+------+
only showing top 3 rows



Calculate the average tip percentage for each combination of sex and smoker.

In [23]:
tips.select(tips.total_bill, tips.tip, tips_pct.alias('tip_pcnt'), tips.sex, tips.smoker).groupby('smoker').pivot('sex').mean('tip_pcnt').show()

+------+-------------------+------------------+
|smoker|             Female|              Male|
+------+-------------------+------------------+
|    No| 0.1569209707691836|0.1606687151291298|
|   Yes|0.18215035269941035|0.1527711752024851|
+------+-------------------+------------------+



Use the seattle weather dataset referenced in the lesson to answer the questions below.

Convert the temperatures to farenheight.\
Which month has the most rain, on average?\
Which year was the windiest?\
What is the most frequent type of weather in January?\
What is the average high and low tempurature on sunny days in July in 2013 and 2014?\
What percentage of days were rainy in q3 of 2015?\
For each year, find what percentage of days it rained (had non-zero precipitation).

In [60]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [31]:
temp_max_f = col('temp_max')*9/5 + 32
temp_min_f = col('temp_min')*9/5 + 32
weather.select(weather.date, weather.precipitation, weather.temp_max, temp_max_f.alias('temp_max_f'),weather.temp_min, temp_max_f.alias('temp_min_f')).show()

+----------+-------------+--------+------------------+--------+------------------+
|      date|precipitation|temp_max|        temp_max_f|temp_min|        temp_min_f|
+----------+-------------+--------+------------------+--------+------------------+
|2012-01-01|          0.0|    12.8|             55.04|     5.0|             55.04|
|2012-01-02|         10.9|    10.6|             51.08|     2.8|             51.08|
|2012-01-03|          0.8|    11.7|             53.06|     7.2|             53.06|
|2012-01-04|         20.3|    12.2|             53.96|     5.6|             53.96|
|2012-01-05|          1.3|     8.9|             48.02|     2.8|             48.02|
|2012-01-06|          2.5|     4.4|             39.92|     2.2|             39.92|
|2012-01-07|          0.0|     7.2|             44.96|     2.8|             44.96|
|2012-01-08|          0.0|    10.0|              50.0|     2.8|              50.0|
|2012-01-09|          4.3|     9.4|             48.92|     5.0|             48.92|
|201

Which month has the most rain, on average?

In [48]:
from pyspark.sql.functions import month, year, quarter


In [55]:
(weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(sum("precipitation").alias("total_rainfall"))
    .sort("month")
    .show())

+-----+------------------+
|month|    total_rainfall|
+-----+------------------+
|    1|465.99999999999994|
|    2|             422.0|
|    3|             606.2|
|    4|             375.4|
|    5|             207.5|
|    6|             132.9|
|    7|              48.2|
|    8|             163.7|
|    9|235.49999999999997|
|   10|             503.4|
|   11|             642.5|
|   12|             622.7|
+-----+------------------+



Which year was the windiest?

In [58]:
(weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(mean("wind").alias("wind_max"))
    .sort("year")
    .show())

+----+------------------+
|year|          wind_max|
+----+------------------+
|2012| 3.400819672131147|
|2013|3.0158904109589044|
|2014|3.3876712328767136|
|2015|  3.15972602739726|
+----+------------------+



What is the most frequent type of weather in January?

In [70]:
( weather.filter(month("date") == 1)).groupby(weather.weather).agg(count('weather')).show()

+-------+--------------+
|weather|count(weather)|
+-------+--------------+
|    fog|            38|
|drizzle|            10|
|   rain|            35|
|    sun|            33|
|   snow|             8|
+-------+--------------+



What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [81]:
(weather.filter(year('date')== 2013)).filter(month('date')== 7).filter(weather.weather == 'sun').agg(mean('temp_max')).show()

+------------------+
|     avg(temp_max)|
+------------------+
|26.585185185185193|
+------------------+



In [82]:
(weather.filter(year('date')== 2013)).filter(month('date')== 7).filter(weather.weather == 'sun').agg(mean('temp_min')).show()

+------------------+
|     avg(temp_min)|
+------------------+
|13.981481481481483|
+------------------+



In [83]:
(weather.filter(year('date')== 2014)).filter(month('date')== 7).filter(weather.weather == 'sun').agg(mean('temp_max')).show()

+-------------+
|avg(temp_max)|
+-------------+
|       27.092|
+-------------+



In [84]:
(weather.filter(year('date')== 2014)).filter(month('date')== 7).filter(weather.weather == 'sun').agg(mean('temp_min')).show()

+------------------+
|     avg(temp_min)|
+------------------+
|14.400000000000002|
+------------------+



#### What percentage of days were rainy in q3 of 2015?

In [177]:
(weather.withColumn("quarter", quarter("date"))).filter(quarter('date') == '3').filter(year('date') == 2015)\
.select(when(col('weather') == 'rain', 1)\
    .otherwise(0).alias('rain')).agg(mean('rain')).show()


+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



#### For each year, find what percentage of days it rained (had non-zero precipitation).

In [153]:
#My way
(weather.withColumn('year', year('date'))).select('year',when(col('precipitation') > 0, 1)\
    .otherwise(0).alias('rain')).groupBy('year')\
        .agg(mean('rain')).show()


+----+-------------------+
|year|          avg(rain)|
+----+-------------------+
|2015|0.39452054794520547|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2012|0.48360655737704916|
+----+-------------------+



In [154]:
# From maggie's notebook
(weather
 .filter(year('date') == 2015)
 .select(when(col('precipitation') > 0, 1).otherwise(0).alias('rain'))
 .agg(mean('rain'))
 .show()
)

+-------------------+
|          avg(rain)|
+-------------------+
|0.39452054794520547|
+-------------------+

